In [0]:
%tensorflow_version 1.x

## Preprocessing:-

In [0]:
import helper
import numpy as np


In [32]:
!unzip 'game-of-thrones-book-files.zip'

Archive:  game-of-thrones-book-files.zip
  inflating: got1.txt                
  inflating: got2.txt                
  inflating: got3.txt                
  inflating: got4.txt                
  inflating: got5.txt                


In [40]:
#Reading the Script
with open("got1.txt",'r') as file:
  got1=file.read()
with open("got2.txt",'r') as file:
  got2=file.read()
with open("got3.txt",'r') as file:
  got3=file.read()
with open("got1.txt",'r') as file:
  got4=file.read()
with open("got1.txt",'r') as file:
  got5=file.read()

text = got1+got2+got3+got4+got5
print(len(text))
print(len(got1))

9702721
1733704


In [41]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 56308
Number of scenes: 42815
Average number of sentences in each scene: 0.0017984351278757444
Number of lines: 42892
Average number of words in each line: 41.90688240231279

The sentences 0 to 10:

The comet’s tail spread across the dawn, a red slash that bled above the crags of Dragonstone like a wound in the pink and purple sky.

The maester stood on the windswept balcony outside his chambers. It was here the ravens came, after long flight. Their droppings speckled the gargoyles that rose twelve feet tall on either side of him, a hellhound and a wyvern, two of the thousand that brooded over the walls of the ancient fortress. When first he came to Dragonstone, the army of stone grotesques had made him uneasy, but as the years passed he had grown used to them. Now he thought of them as old friends. The three of them watched the sky together with foreboding.

The maester did not believe in omens. And yet . . . old as he was, Cressen had

In [0]:

def create_lookup_tables(text):
      #text is split into words
      words = sorted(list(set(text)))
      vocab_to_int = {word:index for index,word in enumerate(words)}
      int_to_vocab = {index:word for index,word in enumerate(words)}
      return vocab_to_int,int_to_vocab


In [0]:
def token_lookup():
    mark_to_token={}
    mark_to_token['.'] = "||Period||"
    mark_to_token[','] = "||Comma||"
    mark_to_token['"'] = "||QuotationMark||"
    mark_to_token[';'] = "||Semicolon||"
    mark_to_token['!'] = "||Exclamationmark||"
    mark_to_token['?'] = "||Questionmark||"
    mark_to_token['('] = "||LeftParentheses||"
    mark_to_token[')'] = "||RightParentheses||"
    mark_to_token['--'] = "||Dash||"
    mark_to_token['\n'] = "||Return||"
    return mark_to_token


In [0]:
def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    import pickle
    """
    Preprocess Text Data
    """
    with open("got1.txt",'r') as file:
      got1=file.read()
    with open("got2.txt",'r') as file:
      got2=file.read()
    with open("got3.txt",'r') as file:
      got3=file.read()
    with open("got1.txt",'r') as file:
      got4=file.read()
    with open("got1.txt",'r') as file:
      got5=file.read()

    text = got1+got2+got3+got4+got5
    

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text)
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))



In [0]:
preprocess_and_save_data("none",token_lookup,create_lookup_tables)


In [47]:
text[:100]

'\nThe comet’s tail spread across the dawn, a red slash that bled above the crags of Dragonstone like '

**Now our checkpoint has been created**

# Building The Netwok

In [0]:
import pickle

int_text, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))

In [0]:
import tensorflow as tf
import numpy as np
import random

In [0]:
def get_inputs():
  Inputs = tf.placeholder(tf.int32,[None,None],name='input')
  Targets = tf.placeholder(tf.int32,[None,None],name='Targets')
  LearningRate= tf.placeholder(tf.float32,name='learningrate')
  return Inputs,Targets,LearningRate

In [0]:
def get_init_cell(batch_size,rnn_size):
  def getCell():
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(rnn_cell,output_keep_prob=0.9)
    return drop

  cell = tf.contrib.rnn.MultiRNNCell([getCell() for i in range(2)])
  initial_state = cell.zero_state(batch_size,tf.float32)
  initial_state =  tf.identity(initial_state,"initial_state")
  return cell,initial_state


In [0]:
def get_embed(input_data,vocab_size,embed_dim):
  embeddings = tf.Variable(tf.truncated_normal([vocab_size,embed_dim]))
  embed = tf.nn.embedding_lookup(embeddings,input_data)
  return embed


In [0]:
def build_rnn(cell,inputs):
  outputs,final_state  = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
  final_state = tf.identity(final_state,name='final_state')

  return (outputs,final_state)


In [0]:
def build_nn(cell,rnn_size,input_data,vocab_size,embed_dim):
  embed = get_embed(input_data,vocab_size,embed_dim)
  (outputs,final_state)  = build_rnn(cell,embed)

  Logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=tf.keras.activations.linear)

  return (Logits,final_state)


In [0]:
def get_batches(int_text,batch_size,seq_len):
  chars_per_batch = batch_size*seq_len
  num_batches = len(int_text)//chars_per_batch
  int_text = int_text[:num_batches*chars_per_batch]
  batches = np.zeros([num_batches,2,batch_size,seq_len])
  counter = 0
  for i in range(batch_size):
      for j in range(0,num_batches):
            batches[j,0,i,:] = int_text[counter:counter+seq_len]
            y_temp = int_text[counter+1:counter+seq_len+1]
            batches[j,1,i,:len(y_temp)] = y_temp            
            counter+=seq_len
  batches[num_batches-1,1,-1,-1] = int_text[0]
  return batches            

In [0]:
#Hyperparameters

num_epochs = 1
batch_size=  256
rnn_size = 300
embed_dims = 300 
seq_len = 42
learning_rate = 0.001
show_every_n_batches = 50

save_dir = './save'



In [0]:
#Building the Graph
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
  vocab_size = len(int_to_vocab)
  input_text,targets,lr = get_inputs()
  input_data_shape = tf.shape(input_text)
  cell,initial_state = get_init_cell(input_data_shape[0],rnn_size)
  Logits,Final_state =   build_nn(cell,rnn_size,input_text,vocab_size,embed_dims)

  probs = tf.nn.softmax(Logits,name='probs')
  #Loss Function
  cost = seq2seq.sequence_loss(Logits,targets,
                               tf.ones([input_data_shape[0],input_data_shape[1]]))
  
  #optimizer
  optimizer = tf.train.AdamOptimizer(lr)

  gradients=optimizer.compute_gradients(cost)

  clipped_gradients = [(tf.clip_by_value(grad,-1.,1.),var) for grad,var in gradients if grad is not None]

  train_op = optimizer.apply_gradients(clipped_gradients)


In [65]:
#Training the Network
batches = get_batches(int_text,batch_size,seq_len)

with tf.Session(graph=train_graph) as sess:
  loader = tf.train.import_meta_graph(load_dir + '.meta')
  loader.restore(sess, load_dir)
  #sess.run(tf.global_variables_initializer())

  for epoch_i in range(num_epochs):
      state = sess.run(initial_state,{input_text: batches[0][0]})
      for batch_i,(x,y) in enumerate(batches):
           feed = {input_text:x,
                  targets:y,
                  lr:learning_rate,
                  initial_state:state}

           train_loss,state,_ = sess.run([cost,Final_state,train_op],feed_dict=feed)
           if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

  # Save Model
  saver = tf.train.Saver()
  saver.save(sess, save_dir)
  print('Model Trained and Saved')


INFO:tensorflow:Restoring parameters from ./save
Epoch   0 Batch    0/205   train_loss = 6.410
Epoch   0 Batch   50/205   train_loss = 6.392


KeyboardInterrupt: ignored

In [0]:
import helper
helper.save_params((seq_len, save_dir))

#pickle.dump((seq_len,save_dir), open('params.p', 'wb'))

In [70]:
#Checkpoint
import tensorflow as tf
import numpy as np
import helper

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

'\nThe comet’s tail spread across the dawn, a red slash that bled above the crags of Dragonstone like '

In [24]:
def get_tensors(loaded_graph):
  with tf.Session(graph=loaded_graph):
      return (tf.get_default_graph().get_tensor_by_name('input:0')
      ,tf.get_default_graph().get_tensor_by_name("initial_state:0")
      ,tf.get_default_graph().get_tensor_by_name('final_state:0')
      ,tf.get_default_graph().get_tensor_by_name('probs:0') )
  
test_get_tensors(get_tensors)

Tests Passed


In [71]:
def pick_word(probabilities,int_to_vocab,top_n=5):
      p  = np.squeeze(probabilities)
      vocab_size = len(int_to_vocab)
      p[np.argsort(p)[:-top_n]] = 0
      p = p / np.sum(p)
      c = np.random.choice(vocab_size, 1, p=p)[0]
      return int_to_vocab[c]
test_pick_word(pick_word)

Tests Passed


In [74]:
gen_length = 200

prime_word = 'comet'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
  loader = tf.train.import_meta_graph(load_dir + '.meta')
  loader.restore(sess, load_dir)

  input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

  gen_sentences = [prime_word+':']
  prev_state = sess.run(initial_state,{input_text:np.array([[1]])})

  for n in range(gen_length):
    #Dynamic Input
        dyn_input = [ [vocab_to_int[word] for word in gen_sentences[-seq_length:]] ]
        dyn_seq_length = len(dyn_input[0])
        probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
  
  tv_script = ' '.join(gen_sentences)
  for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
  tv_script = tv_script.replace('\n ', '\n')
  tv_script = tv_script.replace('( ', '(')
        
  print(tv_script)



INFO:tensorflow:Restoring parameters from ./save


KeyError: ignored

# Test

In [0]:
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.contrib import rnn


def _print_success_message():
    print('Tests Passed')


def test_create_lookup_tables(create_lookup_tables):
    with tf.Graph().as_default():
        test_text = '''
        Moe_Szyslak Moe's Tavern Where the elite meet to drink
        Bart_Simpson Eh yeah hello is Mike there Last name Rotch
        Moe_Szyslak Hold on I'll check Mike Rotch Mike Rotch Hey has anybody seen Mike Rotch lately
        Moe_Szyslak Listen you little puke One of these days I'm gonna catch you and I'm gonna carve my name on your back with an ice pick
        Moe_Szyslak Whats the matter Homer You're not your normal effervescent self
        Homer_Simpson I got my problems Moe Give me another one
        Moe_Szyslak Homer hey you should not drink to forget your problems
        Barney_Gumble Yeah you should only drink to enhance your social skills'''

        test_text = test_text.lower()
        test_text = test_text.split()

        vocab_to_int, int_to_vocab = create_lookup_tables(test_text)

        # Check types
        assert isinstance(vocab_to_int, dict),\
            'vocab_to_int is not a dictionary.'
        assert isinstance(int_to_vocab, dict),\
            'int_to_vocab is not a dictionary.'

        # Compare lengths of dicts
        assert len(vocab_to_int) == len(int_to_vocab),\
            'Length of vocab_to_int and int_to_vocab don\'t match. ' \
            'vocab_to_int is length {}. int_to_vocab is length {}'.format(len(vocab_to_int), len(int_to_vocab))

        # Make sure the dicts have the same words
        vocab_to_int_word_set = set(vocab_to_int.keys())
        int_to_vocab_word_set = set(int_to_vocab.values())

        assert not (vocab_to_int_word_set - int_to_vocab_word_set),\
            'vocab_to_int and int_to_vocab don\'t have the same words.' \
            '{} found in vocab_to_int, but not in int_to_vocab'.format(vocab_to_int_word_set - int_to_vocab_word_set)
        assert not (int_to_vocab_word_set - vocab_to_int_word_set),\
            'vocab_to_int and int_to_vocab don\'t have the same words.' \
            '{} found in int_to_vocab, but not in vocab_to_int'.format(int_to_vocab_word_set - vocab_to_int_word_set)

        # Make sure the dicts have the same word ids
        vocab_to_int_word_id_set = set(vocab_to_int.values())
        int_to_vocab_word_id_set = set(int_to_vocab.keys())

        assert not (vocab_to_int_word_id_set - int_to_vocab_word_id_set),\
            'vocab_to_int and int_to_vocab don\'t contain the same word ids.' \
            '{} found in vocab_to_int, but not in int_to_vocab'.format(vocab_to_int_word_id_set - int_to_vocab_word_id_set)
        assert not (int_to_vocab_word_id_set - vocab_to_int_word_id_set),\
            'vocab_to_int and int_to_vocab don\'t contain the same word ids.' \
            '{} found in int_to_vocab, but not in vocab_to_int'.format(int_to_vocab_word_id_set - vocab_to_int_word_id_set)

        # Make sure the dicts make the same lookup
        missmatches = [(word, id, id, int_to_vocab[id]) for word, id in vocab_to_int.items() if int_to_vocab[id] != word]

        assert not missmatches,\
            'Found {} missmatche(s). First missmatch: vocab_to_int[{}] = {} and int_to_vocab[{}] = {}'.format(
                len(missmatches),
                *missmatches[0])

        assert len(vocab_to_int) > len(set(test_text))/2,\
            'The length of vocab seems too small.  Found a length of {}'.format(len(vocab_to_int))

    _print_success_message()


def test_get_batches(get_batches):
    with tf.Graph().as_default():
        test_batch_size = 128
        test_seq_length = 5
        test_int_text = list(range(1000*test_seq_length))
        batches = get_batches(test_int_text, test_batch_size, test_seq_length)

        # Check type
        assert isinstance(batches, np.ndarray),\
            'Batches is not a Numpy array'

        # Check shape
        assert batches.shape == (7, 2, 128, 5),\
            'Batches returned wrong shape.  Found {}'.format(batches.shape)

        for x in range(batches.shape[2]):
            assert np.array_equal(batches[0,0,x], np.array(range(x * 35, x * 35 + batches.shape[3]))),\
                'Batches returned wrong contents. For example, input sequence {} in the first batch was {}'.format(x, batches[0,0,x])
            assert np.array_equal(batches[0,1,x], np.array(range(x * 35 + 1, x * 35 + 1 + batches.shape[3]))),\
                'Batches returned wrong contents. For example, target sequence {} in the first batch was {}'.format(x, batches[0,1,x])


        last_seq_target = (test_batch_size-1) * 35 + 31
        last_seq = np.array(range(last_seq_target, last_seq_target+ batches.shape[3]))
        last_seq[-1] = batches[0,0,0,0]

        assert np.array_equal(batches[-1,1,-1], last_seq),\
            'The last target of the last batch should be the first input of the first batch. Found {} but expected {}'.format(batches[-1,1,-1], last_seq)

    _print_success_message()


def test_tokenize(token_lookup):
    with tf.Graph().as_default():
        symbols = set(['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n'])
        token_dict = token_lookup()

        # Check type
        assert isinstance(token_dict, dict), \
            'Returned type is {}.'.format(type(token_dict))

        # Check symbols
        missing_symbols = symbols - set(token_dict.keys())
        unknown_symbols = set(token_dict.keys()) - symbols

        assert not missing_symbols, \
            'Missing symbols: {}'.format(missing_symbols)
        assert not unknown_symbols, \
            'Unknown symbols: {}'.format(unknown_symbols)

        # Check values type
        bad_value_type = [type(val) for val in token_dict.values() if not isinstance(val, str)]

        assert not bad_value_type,\
            'Found token as {} type.'.format(bad_value_type[0])

        # Check for spaces
        key_has_spaces = [k for k in token_dict.keys() if ' ' in k]
        val_has_spaces = [val for val in token_dict.values() if ' ' in val]

        assert not key_has_spaces,\
            'The key "{}" includes spaces. Remove spaces from keys and values'.format(key_has_spaces[0])
        assert not val_has_spaces,\
            'The value "{}" includes spaces. Remove spaces from keys and values'.format(val_has_spaces[0])

        # Check for symbols in values
        symbol_val = ()
        for symbol in symbols:
            for val in token_dict.values():
                if symbol in val:
                    symbol_val = (symbol, val)

        assert not symbol_val,\
            'Don\'t use a symbol that will be replaced in your tokens. Found the symbol {} in value {}'.format(*symbol_val)

    _print_success_message()


def test_get_inputs(get_inputs):
    with tf.Graph().as_default():
        input_data, targets, lr = get_inputs()

        # Check type
        assert input_data.op.type == 'Placeholder',\
            'Input not a Placeholder.'
        assert targets.op.type == 'Placeholder',\
            'Targets not a Placeholder.'
        assert lr.op.type == 'Placeholder',\
            'Learning Rate not a Placeholder.'

        # Check name
        assert input_data.name == 'input:0',\
            'Input has bad name.  Found name {}'.format(input_data.name)

        # Check rank
        input_rank = 0 if input_data.get_shape() == None else len(input_data.get_shape())
        targets_rank = 0 if targets.get_shape() == None else len(targets.get_shape())
        lr_rank = 0 if lr.get_shape() == None else len(lr.get_shape())

        assert input_rank == 2,\
            'Input has wrong rank.  Rank {} found.'.format(input_rank)
        assert targets_rank == 2,\
            'Targets has wrong rank. Rank {} found.'.format(targets_rank)
        assert lr_rank == 0,\
            'Learning Rate has wrong rank. Rank {} found'.format(lr_rank)

    _print_success_message()


def test_get_init_cell(get_init_cell):
    with tf.Graph().as_default():
        test_batch_size_ph = tf.placeholder(tf.int32, [])
        test_rnn_size = 256

        cell, init_state = get_init_cell(test_batch_size_ph, test_rnn_size)

        # Check type
        assert isinstance(cell, tf.contrib.rnn.MultiRNNCell),\
            'Cell is wrong type.  Found {} type'.format(type(cell))

        # Check for name attribute
        assert hasattr(init_state, 'name'),\
            'Initial state doesn\'t have the "name" attribute.  Try using `tf.identity` to set the name.'

        # Check name
        assert init_state.name == 'initial_state:0',\
            'Initial state doesn\'t have the correct name. Found the name {}'.format(init_state.name)

    _print_success_message()


def test_get_embed(get_embed):
    with tf.Graph().as_default():
        embed_shape = [50, 5, 256]
        test_input_data = tf.placeholder(tf.int32, embed_shape[:2])
        test_vocab_size = 27
        test_embed_dim = embed_shape[2]

        embed = get_embed(test_input_data, test_vocab_size, test_embed_dim)

        # Check shape
        assert embed.shape == embed_shape,\
            'Wrong shape.  Found shape {}'.format(embed.shape)

    _print_success_message()


def test_build_rnn(build_rnn):
    with tf.Graph().as_default():
        test_rnn_size = 256
        test_rnn_layer_size = 2
        test_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(test_rnn_size) for _ in range(test_rnn_layer_size)])

        test_inputs = tf.placeholder(tf.float32, [None, None, test_rnn_size])
        outputs, final_state = build_rnn(test_cell, test_inputs)

        # Check name
        assert hasattr(final_state, 'name'),\
            'Final state doesn\'t have the "name" attribute.  Try using `tf.identity` to set the name.'
        assert final_state.name == 'final_state:0',\
            'Final state doesn\'t have the correct name. Found the name {}'.format(final_state.name)

        # Check shape
        assert outputs.get_shape().as_list() == [None, None, test_rnn_size],\
            'Outputs has wrong shape.  Found shape {}'.format(outputs.get_shape())
        assert final_state.get_shape().as_list() == [test_rnn_layer_size, 2, None, test_rnn_size],\
            'Final state wrong shape.  Found shape {}'.format(final_state.get_shape())

    _print_success_message()


def test_build_nn(build_nn):
    with tf.Graph().as_default():
        test_input_data_shape = [128, 5]
        test_input_data = tf.placeholder(tf.int32, test_input_data_shape)
        test_rnn_size = 256
        test_embed_dim = 300
        test_rnn_layer_size = 2
        test_vocab_size = 27
        test_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(test_rnn_size) for _ in range(test_rnn_layer_size)])

        logits, final_state = build_nn(test_cell, test_rnn_size, test_input_data, test_vocab_size, test_embed_dim)

        # Check name
        assert hasattr(final_state, 'name'), \
            'Final state doesn\'t have the "name" attribute.  Are you using build_rnn?'
        assert final_state.name == 'final_state:0', \
            'Final state doesn\'t have the correct name. Found the name {}. Are you using build_rnn?'.format(final_state.name)

        # Check Shape
        assert logits.get_shape().as_list() == test_input_data_shape + [test_vocab_size], \
            'Outputs has wrong shape.  Found shape {}'.format(logits.get_shape())
        assert final_state.get_shape().as_list() == [test_rnn_layer_size, 2, 128, test_rnn_size], \
            'Final state wrong shape.  Found shape {}'.format(final_state.get_shape())

    _print_success_message()


def test_get_tensors(get_tensors):
    test_graph = tf.Graph()
    with test_graph.as_default():
        test_input = tf.placeholder(tf.int32, name='input')
        test_initial_state = tf.placeholder(tf.int32, name='initial_state')
        test_final_state = tf.placeholder(tf.int32, name='final_state')
        test_probs = tf.placeholder(tf.float32, name='probs')

    input_text, initial_state, final_state, probs = get_tensors(test_graph)

    # Check correct tensor
    assert input_text == test_input,\
        'Test input is wrong tensor'
    assert initial_state == test_initial_state, \
        'Initial state is wrong tensor'
    assert final_state == test_final_state, \
        'Final state is wrong tensor'
    assert probs == test_probs, \
        'Probabilities is wrong tensor'

    _print_success_message()


def test_pick_word(pick_word):
    with tf.Graph().as_default():
        test_probabilities = np.array([0.1, 0.8, 0.05, 0.05])
        test_int_to_vocab = {word_i: word for word_i, word in enumerate(['this', 'is', 'a', 'test'])}

        pred_word = pick_word(test_probabilities, test_int_to_vocab)

        # Check type
        assert isinstance(pred_word, str),\
            'Predicted word is wrong type. Found {} type.'.format(type(pred_word))

        # Check word is from vocab
        assert pred_word in test_int_to_vocab.values(),\
            'Predicted word not found in int_to_vocab.'


    _print_success_message()

